In [1]:
%run 'InitializationFullRatings.ipynb'

6040
['2', 'M', '56', '16', '70072']
3883
['1', 'Toy Story', "Animation|Children's|Comedy", '1995']
(3884L, 6041L)
98742
[594, 1]
(3884L, 6041L)
69
3883
3
0
0
5
1
18
(3884L, 19L)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.]
2.91547594742
(3884L, 6041L)
([3, 45, 7, 2], [2, 54, 13, 15], 0.97228425171235)
(6041L, 19L)


In [2]:
clf = linear_model.LinearRegression()

In [16]:
print training_ratings_full.shape
initial_model_parameter.shape

(3884L, 6041L)


(6041L, 19L)

In [17]:
def feature_cleaning(features,original):
    for i in range(len(original)):
        for j in range(len(original[i])):
            if(original[i][j]==0):
                features[i][j]=0
    return features

In [18]:
#Calculating movie features
def collaboration_filtering(ratings,features,model, steps, alpha, lambd):
    original=np.copy(features)
    for i in range(steps):
        error=np.transpose(np.dot(model,np.transpose(features))) - ratings
        val=np.dot(error,model) + (lambd * features)
        features=features-alpha*val
        features=feature_cleaning(features,original)
    return features

In [20]:
features=collaboration_filtering(training_ratings_full,movies_genre_array,initial_model_parameter, 1, .002, 1.5)

In [8]:
rat=np.dot(features,np.transpose(initial_model_parameter))

In [9]:
print calcRMSE(np.transpose(rat),validation_ratings_full,validation_matrix_full)
from sklearn.metrics import mean_squared_error
import math
print math.sqrt(mean_squared_error(rat, full_ratings))

1.55230862451
0.622516427168


In [10]:
#Calculating hybrid function
def hybrid(ratings,features,model, steps, alpha, lambd):
    original=np.copy(features)
    for i in range(steps):
        error=np.transpose(np.dot(model,np.transpose(features))) - ratings
        feature_grad=np.dot(error,model) + (lambd * features)
        model_grad=np.dot(np.transpose(error),features) + (lambd * model)
        features=features-alpha*feature_grad
        model=model-alpha*model_grad
        features=feature_cleaning(features,original)
    return features,model

In [11]:
final_features,model=hybrid(training_ratings_full,features,initial_model_parameter, 10, .000002, 1.5)

In [12]:
rat=np.dot(final_features,np.transpose(model))

In [13]:
print calcRMSE(np.transpose(rat),validation_ratings_full,validation_matrix_full)
print math.sqrt(mean_squared_error(rat, full_ratings))

1.54895023056
0.384530678807


In [14]:
model[5998]

array([-0.00030009,  0.07202612,  0.04310627,  0.02272406,  0.03769986,
        0.17070021,  0.03680096,  0.02372642,  0.23670963,  0.01088779,
        0.03195384,  0.04657753,  0.02894887,  0.02696078,  0.07372316,
        0.04248296,  0.07391841,  0.03161089,  0.03405603])

In [188]:
movies_genre_array[100]

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.])

In [52]:
movies_genre_testing_array = np.array([[0.0 for i in range(len(movie_types)+1)] for j in range(len(testing_movies)+1)])
print movies_genre_testing_array.shape

(70L, 19L)


In [53]:
#Calculating genre matrix for new movies
count=1
for m in movies:
    if int(m[0])>len(movies):      
        genre_list=m[2].split("|")
        for t in m[2].split("|"):
            index=movie_types.get(t)
            movies_genre_testing_array[count,index]=10
        count+=1

In [68]:
f=np.dot(movies_genre_testing_array,np.transpose(model))

In [67]:
testing_ratings = np.array([[0 for i in range(len(users)+1)] for j in range(len(testing_movies)+1)])
print testing_ratings.shape

(70L, 6041L)


In [59]:
#Fetching testing ratings
for line in open('ratings.dat', 'r'):
    temp= line.rstrip().split("::")
    if int(temp[1])>len(movies):
        a=int(temp[1])-3883
        testing_ratings[a,temp[0]]=temp[2]

In [63]:
testing_ratings.shape

(70L, 6041L)

In [69]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(testing_ratings, f))

In [70]:
rms

2.215991086306548